In [1]:
import bs4
import time
import platform
from datetime import datetime

In [2]:
import get_information

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.keys import Keys

In [4]:
# INSTANCE BROWSER
def openBrowser(business):
    '''
        Verifica qual o sistema operacional para que use o driver e diretorio de usuarios certo
    '''
    
    if platform.system() == 'Linux':
        driver_path = './webdriver/linux/chromedriver'
        profile_path = '/home/nomadbitcoin/whatsapp_profiles/' + business 
    elif platform.system() == 'Windows':
        path = './webdriver/win/chromedriver.exe'
        profile_path = 'Need Implement'
    
    
    global driver
    try:
        options = webdriver.ChromeOptions()
        # open with saved informations in cache
        options.add_argument("profile")
        options.add_argument("user-data-dir=" + profile_path)
        driver = webdriver.Chrome(options=options, executable_path=driver_path)
#         driver.implicitly_wait(5) #tempo implicito de espera 1 segundo antes de cada acao
        main_url = 'https://web.whatsapp.com/'
        driver.get(main_url)
        if wait_connection(driver):
            return True, 'opened in: {}'.format(main_url)
    except Exception as error:
        print(type(error), error)
        return False
    
# PARA LIBERAR SOMENTE APOS TER ABERTO O WHATSAPP
def wait_connection(driver):
    waiting = True
    while waiting:
        try:
            if driver.find_element_by_class_name('Qk8nZ').text == 'Keep your phone connected':
                waiting = False
                return True
                break
#             elif driver.find_element_by_class_name('_13HPh').text == 'Computer not connected':
#                 waiting = False
#                 return False, 'Computer not connected'
        except KeyboardInterrupt:
            waiting = False
            return False
        except:
            time.sleep(10)
            pass

In [5]:
openBrowser('nomadbitcoin')

(True, 'opened in: https://web.whatsapp.com/')

In [6]:
gdata = get_information.get_Data(driver)

In [7]:
chats = gdata.getChats()

In [8]:
def getPageSource():
    global htmlPage
    htmlPage = gdata.getPageSource(True)

In [16]:
messages = gdata.slc_get_messages()
time.sleep(1)
print(gdata.msg_type_content(messages[-1]))

link


In [ ]:
def slc_get_last_msg():
    '''
        Pega a ultima mensagem do historico, o tipo do conteudo, horario e status
    '''
    getPageSource()
    messages = gdata.slc_get_messages()
    msg_time = gdata.getTime(messages[-1], 'message')
    return messages[-1]

In [ ]:
slc_get_last_msg()